In [28]:
import time
import math
import random
import numpy as np

import spacy
spacy.load('fr_core_news_sm')
spacy.load('en_core_web_sm')

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

In [30]:
spacy_fr = spacy.load("fr_core_news_sm")
print(len(spacy_fr.vocab))
spacy_en = spacy.load("en_core_web_sm")
print(len(spacy_en.vocab))

424
761


In [31]:
def tokenize_fr(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)][::-1]

In [32]:
SRC = Field(tokenize=tokenize_fr, 
            init_token="<sos>", 
            eos_token="<eos>", 
            lower=True)

TRG = Field(tokenize=tokenize_en, 
            init_token="<sos>", 
            eos_token="<eos>", 
            lower=True)

print(type(SRC), type(TRG))

<class 'torchtext.data.field.Field'> <class 'torchtext.data.field.Field'>


In [33]:
train_data, valid_data, test_data = Multi30k.splits(path="/home/soohyun/data/multi30k/",\
                                                    exts=(".fr",".en"), fields=(SRC, TRG))

print('train:',len(train_data))
print('valid:',len(valid_data))
print('test:',len(test_data))

train: 29000
valid: 1014
test: 1000


In [34]:
print(vars(train_data.examples[0])) 

{'src': ['.', 'buissons', 'de', 'près', 'dehors', 'sont', 'blancs', 'hommes', 'jeunes', 'deux'], 'trg': ['.', 'bushes', 'many', 'near', 'outside', 'are', 'males', 'white', ',', 'young', 'two']}


In [35]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [36]:
print('len of French vocab:',len(SRC.vocab))
print('len of English vocab:',len(TRG.vocab))

len of French vocab: 6470
len of English vocab: 5893


In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cuda


In [38]:
BATCH_SIZE = 50

train_iter, valid_iter, test_iter = BucketIterator.splits((train_data, valid_data, test_data),
                                                         batch_size=BATCH_SIZE,
                                                         device=device)


print(len(train_iter), len(valid_iter), len(test_iter))

580 21 20


In [39]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        print('[enc]hid dim:', hid_dim)
        self.hid_dim = hid_dim
        
        self.n_layers = n_layers
        print('[enc]n layers:', n_layers)
        self.embedding = nn.Embedding(input_dim, emb_dim)
        print('[enc]embedding (input_dim, emb_dim):', (input_dim, emb_dim))
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        print('[enc]LSTM(input_dim, emb_dim, n layers):', (input_dim, emb_dim, n_layers))
        print('[enc]dropout:', dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        print('[enc]embedded size:', embedded.size())
        outputs, (hidden, cell) = self.rnn(embedded)
        print('[enc]outputs size:', outputs.size())
        print('[enc]hidden size:', hidden.size())
        print('[enc]cell size:', cell.size())
        return hidden, cell

In [40]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        print('[dec]output dim:', output_dim)
        self.output_dim = output_dim
        print('[dec]hid dim:', hid_dim)
        self.hid_dim = hid_dim
        print('[dec]n layers:', n_layers)
        self.n_layers = n_layers
        print('[dec]embedding (output_dim, emb_dim):', (output_dim, emb_dim))
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        print('[dec]LSTM (emb_dim, hid_dim, n layers):', (emb_dim, hid_dim, n_layers))
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        print('[enc]nn.Linear (hid_dim, output_dim):', (hid_dim, output_dim))
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        print('[enc]input size:', input.size())
        embedded = self.dropout(self.embedding(input))
        print('[enc]input size:', embedded.size())
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        print('[enc]prediction size:', prediction.size())
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

In [41]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert encoder.hid_dim == decoder.hid_dim, \
        "hidden dimensions of encoder and decoder must be equal"
        assert encoder.n_layers == decoder.n_layers, \
        "encoder and decoder must have equal number of layers"
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # last hidden state of the encoder is used as the init hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        # first input to the decoder is the <sos> token
        input = trg[0,:]
        
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            # receive output tensor (predictions) and new hidden and cell states
            outputs[t] = output
            
            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            # get the highest predicted token from out predictions
            top1 = output.argmax(1)
            
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1
            
        return outputs

In [42]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

[enc]hid dim: 512
[enc]n layers: 2
[enc]embedding (input_dim, emb_dim): (6470, 256)
[enc]LSTM(input_dim, emb_dim, n layers): (6470, 256, 2)
[enc]dropout: 0.5
[dec]output dim: 5893
[dec]hid dim: 512
[dec]n layers: 2
[dec]embedding (output_dim, emb_dim): (5893, 256)
[dec]LSTM (emb_dim, hid_dim, n layers): (256, 512, 2)
[enc]nn.Linear (hid_dim, output_dim): (512, 5893)


In [43]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6470, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [44]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model)} trainable parameters")

The model has 13544453 trainable parameters


In [45]:
optimizer = optim.Adam(model.parameters())

In [46]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [47]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        output = model(src, trg)
        
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
    return epoch_loss/len(iterator)

In [48]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i,batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            
            output = model(src, trg, 0)  # turn off teacher forcing
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
            
    return epoch_loss/len(iterator)

In [49]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time/60)
    elapsed_secs = int(elapsed_time - (elapsed_mins *60))
    return elapsed_mins, elapsed_secs

In [50]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)
    print('valid loss:', valid_loss)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "fr2en.pt")
        
    print(f"Epoch: {epoch+1:2} | Time: {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}")
    print(f"\tVal. loss: {valid_loss:.3f} | Val. PPL: {math.exp(valid_loss):7.3f}")

[enc]embedded size: torch.Size([26, 50, 256])
[enc]outputs size: torch.Size([26, 50, 512])
[enc]hidden size: torch.Size([2, 50, 512])
[enc]cell size: torch.Size([2, 50, 512])
[enc]input size: torch.Size([1, 50])
[enc]input size: torch.Size([1, 50, 256])


/home/soohyun/anaconda3/envs/torch/lib/python3.9/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


UnboundLocalError: local variable 'prediction' referenced before assignment

In [ ]:
model.load_state_dict(torch.load('fr2en.pt'))
test_loss = evaluate(model, test_iter, criterion)
print(f"Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f}")